In [40]:
import os

In [37]:

os.environ['OPENAI_API_KEY'] = os.environ.get('OPENAI_API_KEY')

from langchain import PromptTemplate,OpenAI,LLMChain

template = "{flower}的话语是？"

llm1 = OpenAI(temperature=0)

llmchain = LLMChain(
    llm=llm,
    prompt = PromptTemplate.from_template(template)
)
#result = llmchain.invoke("玫瑰")
#print(result)

# apply允许您针对输入列表运行链
input_list = [ 
    {"flower": "玫瑰",'season': "夏季"},
    {"flower": "百合",'season': "春季"}, 
    {"flower": "郁金香",'season': "秋季"}]
result = llmchain.apply(input_list)
print(result)

[{'text': '\n\n"你是我心中最美的花朵，永远都散发着迷人的芬芳，让我为你倾尽一生的爱。"'}, {'text': '\n\n百合的话语是“我爱你”。'}, {'text': '\n\n"我是爱情的象征，美丽而短暂，但永远留在你的心中。"'}]


In [48]:
from langchain.chains import SequentialChain
llm1 = OpenAI(temperature=0.7)
template = """
你是一个植物学家。给定花的名称和类型，你需要为这种花写一个100字左右的介绍。

花名：{name}
颜色:{color}
植物学家：这是关于上述花的介绍"
"""

prompt_template = PromptTemplate(
    input_variables=["name","color"],template=template
)

introduction_chain = LLMChain(llm=llm1,prompt=prompt_template,output_key="introduction")
#introduction_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="introduction")

In [49]:
#2
llm2 = OpenAI(temperature=0.7)
review_tempate = """
你是一位鲜花评论家，给定一种鲜花的介绍。你需要为这种花写一篇100字左右的评论。

鲜花介绍
{introduction}
花评人对上述花的评论: """
prompt_template = PromptTemplate(
    input_variables=["introduction"],template=template
)
review_chain = LLMChain(llm=llm2,prompt=prompt_template,output_key="review")

#3
llm3 = OpenAI(temperature=0.7)

social_tempate = """
你是一家花店的运营经理。给定一种花的介绍和评论。你需要为这种花写一篇200字左右的社交媒体文章。

鲜花介绍
{introduction}
花评人对上述花的评论: 
{review}

社交媒体文章：
"""
prompt_template = PromptTemplate(
    input_variables=["introduction","review"],template=template
)
social_post_chain = LLMChain(llm=llm3,prompt=prompt_template,output_key="social_post_text")

In [50]:
overall_chain = SequentialChain(
    chains=[introduction_chain,review_chain,social_post_chain],
    input_variables=["name","color"],
    output_variables=["introduction","review","social_post_text"]
)

result = overall_chain({"name":"玫瑰","color":"黑色"})
print(result)

{'name': '玫瑰', 'color': '黑色', 'introduction': '\n玫瑰是一种美丽的花朵，它的花瓣盛开时散发着迷人的香气。它的颜色通常是红色或粉色，但也有一些特殊的品种，如黑色玫瑰。黑色玫瑰是一种罕见的珍贵品种，它象征着神秘和浪漫。它的花瓣光滑而柔软，给人一种温柔的感觉。它的外观和香气吸引了无数人的眼球，被用来表达爱情和感谢。在自然界，黑色玫瑰也是一种难得的奇迹，它的生长需要特殊的环境和护理。作为植物学家，我对玫瑰的美丽和生长过程', 'review': '\n玫瑰是一种广泛种植的美丽花卉，其优雅的花形和丰富的色彩令人难以抗拒。其中，黑色玫瑰更是独具魅力，它代表着神秘和典雅。黑色玫瑰属于蔷薇科，是多年生木质植物。它的花朵呈球形，花瓣紧密排列，花蕊隐约可见。黑色玫瑰的叶子呈羽状复叶，叶片有光泽，株高可达1米。它喜欢温暖湿润的气候，适宜生长在肥沃的土壤中。在花语中，黑色玫瑰象征着热情和沉默，也是', 'social_post_text': '\n玫瑰是一种美丽的花卉，它的颜色多样，包括红色、粉色、白色和黄色等。但在所有的花色中，黑色玫瑰无疑是最特别的一种。黑色玫瑰具有神秘而优雅的气质，它的花瓣如丝绸般柔软，绽放出迷人的光芒。它的花语是神秘和热情，代表着深深的爱和浪漫的情怀。黑色玫瑰在鲜花市场上备受欢迎，常被用于表达对心爱之人的感情。它也是许多婚礼上必不可少的花朵，代表着永恒的爱情和忠诚。无论是作为'}


#注意链里的每个大模型实例，可以是同一个实例，也可以是不同的实例。

In [52]:
# 构建两个场景的模板
flower_care_template = """你是一个经验丰富的园丁，擅长解答关于养花育花的问题。
                        下面是需要你来回答的问题:
                        {input}"""

flower_deco_template = """你是一位网红插花大师，擅长解答关于鲜花装饰的问题。
                        下面是需要你来回答的问题:
                        {input}"""

# 构建提示信息
prompt_infos = [
    {
        "key": "flower_care",
        "description": "适合回答关于鲜花护理的问题",
        "template": flower_care_template,
    },
    {
        "key": "flower_decoration",
        "description": "适合回答关于鲜花装饰的问题",
        "template": flower_deco_template,
    }]

In [54]:
llm = OpenAI()
chain_map={}
for info in prompt_infos:
    prompt = PromptTemplate(
        template = info["template"],
        input_variable=["input"]
    )
    print("目标提示"+info["key"])
    chain = LLMChain(llm=llm,prompt=prompt,verbose=True)
    chain_map[info["key"]]= chain

目标提示flower_care
目标提示flower_decoration


In [60]:
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE as RouterTemplate

destinations =[f"{p['key']}:{p['description']}" for p in prompt_infos]
router_template = RouterTemplate.format(destinations="\n".join(destinations))
print("路由模版:\n",router_template)

路由模版:
 Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
flower_care:适合回答关于鲜花护理的问题
flower_decoration:适合回答关于鲜花装饰的问题

<< INPUT >>
{input}

<< OUTPUT (mus

In [62]:
router_prompt= PromptTemplate(
    template = router_template,
    input_variables=["input"],
    output_parser = RouterOutputParser()
)
print("路由提示\n",router_prompt)

router_chain = LLMRouterChain.from_llm(llm,router_prompt,verbose=True)

路由提示
 input_variables=['input'] output_parser=RouterOutputParser() template='Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object formatted to look like:\n```json\n{{\n    "destination": string \\ name of the prompt to use or "DEFAULT"\n    "next_inputs": string \\ a potentially modified version of the original input\n}}\n```\n\nREMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.\nREMEMBER: "next_inputs" can just be the original input if you don\'t think any modifications are needed.\n\n<< CANDIDATE PROMPTS >>\nflower

In [63]:
from langchain.chains import ConversationChain
default_chain = ConversationChain(llm=llm,output_key="text",verbose=True)

In [65]:
from langchain.chains.router import MultiPromptChain
chain = MultiPromptChain(
    router_chain = router_chain,
    destination_chains = chain_map,
    default_chain=default_chain,
    verbose=True
)
print(chain.invoke("如何为玫瑰花浇水？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
flower_care: {'input': '如何为玫瑰花浇水？'}

> Entering new LLMChain chain...
Prompt after formatting:
你是一个经验丰富的园丁，擅长解答关于养花育花的问题。
                        下面是需要你来回答的问题:
                        如何为玫瑰花浇水？

> Finished chain.

> Finished chain.
{'input': '如何为玫瑰花浇水？', 'text': '\n\n为玫瑰花浇水需要注意以下几点：\n\n1. 浇水的频率：一般情况下，玫瑰花需要每周浇水2-3次，但具体的浇水频率还要根据当地的气候、温度和土壤湿度来决定。在天气炎热干燥的时候，可能需要每天浇水。\n\n2. 浇水的时间：最佳的浇水时间是在早晨或者傍晚，避免在太阳最强烈的时候浇水，以免水滴在叶面上引起叶子烧伤。\n\n3. 浇水的方式：可以使用浇水壶或者喷壶来浇水，让水流缓慢、均匀地淋在土壤上。避免直接浇水到花'}


In [66]:
print(chain.invoke("如何考入清华大学？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
None: {'input': '如何考入清华大学？'}

> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 如何考入清华大学？
AI:

> Finished chain.

> Finished chain.
{'input': '如何考入清华大学？', 'history': '', 'text': ' 要考入清华大学，首先你需要参加高考，通过高考后才能考入清华大学。清华大学是一所位于北京的顶尖大学，它的历史可以追溯到1911年。每年清华大学都会招收来自全国各地的优秀学生，但是入学难度非常高。通常需要在高考中获得非常优秀的成绩，同时还要通过清华大学的招生考试和面试。如果你有兴趣考入清华大学，建议你提前准备，多加练习，努力提高自己的学习成绩。'}


In [68]:
print(chain.invoke("如何为婚礼现场布置花饰？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
flower_decoration: {'input': '如何布置花饰?'}

> Entering new LLMChain chain...
Prompt after formatting:
你是一位网红插花大师，擅长解答关于鲜花装饰的问题。
                        下面是需要你来回答的问题:
                        如何布置花饰?

> Finished chain.

> Finished chain.
{'input': '如何布置花饰?', 'text': '\n布置花饰的方法有很多种，可以根据不同的场合和需求来选择合适的方式。以下是一些常用的布置花饰的方法：\n\n1. 花束：将不同种类或颜色的鲜花捆绑在一起，形成一个小束，并用绳子或丝带绑紧。可以放在花瓶中，也可以直接手持或挂在墙上，适合用于婚礼、生日等场合。\n\n2. 花篮：将鲜花放在篮子中，可以根据需要加上一些绿叶或装饰物，可以作为中心装饰放在桌子上或用来装饰门口。\n\n3. 花环：将鲜花或干花串成一个圆形，可以悬挂在门口或墙上，也可以放在桌子上'}
